# pydantic
framework to defining specialized python classes  
classes attributes a called fields  
specialized classe is called model  
simple way to load data from a dic or json into a model (python class) - deserializing data  
simple way to extract model instance data to dic or json - serializing model instance  
validation data during deserialization  
pydantic v2 is written in rust  
data classes not provide deserialization and serialization, but it is fast than pydantic   


# dataclasses
is a code generator, create a brand new class with all the functionality

In [1]:
from dataclasses import dataclass

In [2]:
@dataclass
class Model:
    language: str
    version: str
    year: int

In [8]:
m_dataclass = Model(language="eng", version="1.0", year=2024)

# pydantic

In [4]:
from pydantic import BaseModel

In [5]:
class ModelPydantic(BaseModel):
    language: str
    version: str
    year: int

In [7]:
m_pydantic = ModelPydantic(language="eng", version="1.0", year=2024)